# Introducing Tools

Great! Our chatbot has the basics of inner workings. But that's boring, so let's go and give it some tools! Tools are what differentiates an AI Assistant to an AI Agent.
Let's keep it simple for this example, but you can make it as complex as you'd like.

Here, we'll define a variable that holds in all of my super secret emails, and the agent will be able to acces that *when it decides it's proper*. That's the magic, the tool is always there, and if the agent feels necessary, it will call on it.

---
In this module specifically, we'll build an agent that, firstly goes thorugh a `router_agent` and decides on what to do, does it need to access the provided `emails`, or just go straight into a `chat` function?

If it decides that it needs to access the `emails`, it will direct the graph to the `email_tool`, which will add the necessary context to the agent state, and then go to chat for the completion of the prompt

If not, it just goes straight to the chat and completes the prompt.

Let's see how we can do it

## 1. Configs

### 1.1 Installs

* `dotenv` – loads and manages environment variables from a `.env` file.

* `langchain-core` – the lightweight core of LangChain, providing the `Runnable` interfaces and core abstractions.

* `langchain_mcp_adapters` – bridges LangChain with MCP servers, letting agents call remote tools securely. (Only needed if dealing with MCP Servers)

* `langgraph` – builds stateful agent workflows as directed graphs of nodes and edges.

* `requests` – simple HTTP client for making API calls (e.g., to Databricks endpoints).

* `truststore` – ensures Python’s SSL connections use your system’s trusted certificate store. (Also only needed if dealing with MCP Servers)

In [ ]:
%%capture
%pip install \
   dotenv \
  "langchain-core==0.3.79" \
  "langchain_mcp_adapters==0.1.11" \
  "langgraph==0.2.41" \
  requests \
  truststore 


In [ ]:
# restart kernel to use newly installed packages
# dbutils.library.restartPython() # Databricks
%reset -f # VSCode

### 1.2 Imports

 - from `langgraph.graph import StateGraph, END`
   - `StateGraph` - build and compile our node graph.
   - `END` - sentinel that tells LangGraph where to stop.

 - `from langchain_core.runnables import RunnableLambda` - wraps a normal Python function so the graph can call it like any other LangChain “runnable.”

 - `from typing import Dict, List, Optional, TypedDict` - creates AgentState, a typed dictionary that documents (and type-checks) the keys we pass between nodes.

 - `my_functions` - Our custom useful functions

 - `os` - Access environment variables that store our Azure configuratio

 - `dotenv.load_dotenv` - Reads the `.env` file and safely loads the environmental variables

 - `json` - Package to manage json-style strings/objects

In [ ]:
# LangGraph core components for building agent workflows
from langgraph.graph import StateGraph, END       # StateGraph: main graph builder, END: termination signal
from langchain_core.runnables import RunnableLambda  # Converts functions to graph-compatible nodes

# Type annotations for better code clarity and IDE support
from typing import Dict, List, Optional, TypedDict

# Utility and system packages
from dotenv import load_dotenv
import os
import json

# Our custom utility functions from the previous notebook
from my_functions import databricks_llm

### 1.3 Loading Environmental Variables

Load our Azure OpenAI configuration from the secure `.env` file.

In [ ]:
# Load environment variables (Azure endpoints, API versions, etc.)
load_dotenv(".env")

## 2 Defining Functions and Classes

### 2.1 Classes

Here, let's change the `AgentState` class a bit:
 - We need to add a few fields; Let's add a field of available tools, so we can check all the tools we have access to at any point in the graph
 - We'll aslo add a field called tool_context, for any new context the tools might give that other nodes can take advantage of.

In [ ]:
class AgentState(TypedDict, total=False):
    """
    Enhanced state container for our tool-enabled agent.
    
    Carries conversation data and tool information between graph nodes.
    """
    # Core conversation data (from previous notebook)
    chat_history: List[Dict[str, str]]        # Complete conversation in OpenAI format
    output: Optional[str]                     # Most recent response
    
    # New fields for tool integration
    available_tools: Optional[Dict[str, str]] # Tool names → descriptions for router
    tool_context: Optional[str]               # Context retrieved by tools (cleared each turn)

### 2.2 Agent Functions with Tool Support

**Three Specialized Agent Functions:**

**1. Router Agent (`router_agent`)**
- **Purpose**: Decides which tool to use based on user's query
- **Input**: Chat history + available tools
- **Output**: JSON decision like `{"tool": "email"}` or `{"tool": "chat"}`
- **Key Feature**: Uses its own system prompt to focus on tool selection

**2. Email Tool (`email_tool`)**  
- **Purpose**: Retrieves email data
- **Process**: Adds results to `tool_context`
- **No LLM needed**: Pure data retrieval function

**3. Enhanced Chat Agent (`chat_agent`)**
- **Purpose**: Generates final response using conversation + tool context
- **Enhancement**: Includes tool context in the prompt for informed responses

#### 2.2.1 - Router Agent

In [ ]:
def router_agent(state):
    """
    Router agent that decides which tool to use based on the user's query.
    
    This agent analyzes the conversation and available tools to make an 
    intelligent routing decision.
    """
    print("\n--- ROUTER AGENT NODE ---")
    print(f"Analyzing user query with {len(state.get('available_tools', {}))} available tools")

    # === BUILD TOOL CATALOG FOR LLM ===
    # Create a formatted list of available tools and their descriptions
    tool_lines = [
        f"- {name}: {desc}"
        for name, desc in (state["available_tools"] or {}).items()
    ]
    tool_catalog = "\n".join(tool_lines) or "none"

    # === CREATE ROUTER-SPECIFIC SYSTEM PROMPT ===
    # The router has a specialized role: tool selection only
    router_system_prompt = (
        "You are an AI router. Choose the single best tool for answering the user's "
        "latest message.\n\n"
        f"Available tools:\n{tool_catalog}\n\n"
        "Return ONLY a JSON object like {\"tool\": \"chat\"} or {\"tool\": \"email\"}."
    )

    # === PREPARE MODIFIED CHAT HISTORY ===
    # Replace system prompts with router-specific prompt
    # This ensures the LLM focuses on tool selection, not general chat
    modified_chat_history = [
        {"role": "system", "content": router_system_prompt}
    ] + [
        m for m in state["chat_history"] 
        if m["role"] != "system"  # Filter out original system prompts
    ]

    print(f"Sending {len(modified_chat_history)} messages to LLM for routing decision")

    # === GET ROUTING DECISION FROM LLM ===
    llm_response = databricks_llm(modified_chat_history, os.getenv("INSTRUCT_ENDPOINT"))
    
    print(f"LLM routing response: {llm_response}")

    # === EXTRACT JSON DECISION ===
    # Parse JSON from LLM response (handle any extra text)
    start = llm_response.rfind("{")
    end   = llm_response.rfind("}")
    
    if start == -1 or end == -1:
        # Fallback to chat if no valid JSON found
        decision = {"tool": "chat"}
        print("⚠️  No valid JSON found, defaulting to chat")
    else:
        decision_json = llm_response[start : end + 1]
        try:
            decision = json.loads(decision_json)
            print(f"✓ Extracted decision: {decision}")
        except json.JSONDecodeError:
            decision = {"tool": "chat"}
            print("⚠️  JSON parse error, defaulting to chat")

    # === UPDATE STATE ===
    # Store decision as JSON string for conditional edges
    state["output"] = json.dumps(decision)

    print("--- ROUTER AGENT NODE END ---\n")
    return state

#### 2.2.2 - Email Tool

In [ ]:
def email_tool(state):
    print("\n--- EMAIL TOOL NODE ---")

    with open("data/mock_emails.json", "r") as f:
        email_archive = json.load(f)

    # Neatly format the email archive into a string
    context = "\n\n".join(
        f"""From: {email['sender']}
        Subject: {email['subject']}
        Body: {email['body']}"""
    for email in email_archive)

    print(f"Email archive context: {context[:100]}...")
    # Store ONLY in scratch space – do not touch chat history
    state["tool_context"] = context
    state["output"] = "email_context_ready"   # optional status message

    print("\n--- EMAIL TOOL NODE END ---")

    return state

#### 2.2.3 Chat Agent

In [ ]:
def chat_agent(state):
    """
    Enhanced chat agent that incorporates tool context into responses.
    
    This agent generates the final response using both the conversation
    history and any additional context provided by tools.
    """
    print("\n--- CHAT AGENT NODE ---")
    
    # === PREPARE ENHANCED CHAT HISTORY ===
    # Start with the original conversation
    enhanced_chat_history = state["chat_history"].copy()
    
    # === ADD TOOL CONTEXT IF AVAILABLE ===
    if state.get("tool_context"):
        print("✓ Including tool context in conversation")
        # Add tool context as a user message so the AI can reference it
        context_message = {
            "role": "user", 
            "content": f"ADDITIONAL CONTEXT FROM TOOLS:\n{state['tool_context']}"
        }
        enhanced_chat_history.append(context_message)
    else:
        print("No tool context available")

    print(f"Sending {len(enhanced_chat_history)} messages to LLM")
    
    # === GENERATE RESPONSE ===
    reply = databricks_llm(enhanced_chat_history, os.getenv("CHAT_ENDPOINT"))
    
    # === UPDATE STATE ===
    # Add only the actual AI response to the permanent chat history
    # (Don't include the temporary tool context message)
    state["chat_history"].append({"role": "assistant", "content": reply})
    state["output"] = reply

    print(f"✓ Generated response: {reply[:100]}...")
    print("--- CHAT AGENT NODE END ---\n")

    return state

## 3 Building the Tool-Enabled Agent Graph

Now we'll create a more sophisticated graph that routes between different capabilities.

### 3.1 Graph with Conditional Routing

**Advanced Graph Features:**

**Conditional Edges:** Unlike simple edges, conditional edges make routing decisions based on the state content.

**Our Graph Flow:**
```
START → router_agent → [decision] → email_tool → chat_agent → END
                           ↓
                       chat_agent → END
```

**Key Concepts:**
- **Entry Point**: `router_agent` (decides the path)
- **Route Selector**: Function that parses the router's JSON decision
- **Conditional Logic**: Routes to `email_tool` or `chat_agent` based on decision
- **Convergence**: Both paths eventually lead to `chat_agent` for final response

In [ ]:
# === STEP 1: INITIALIZE GRAPH ===
g = StateGraph(AgentState)
print("✓ Graph initialized with enhanced AgentState")

# === STEP 2: ADD NODES ===
# Add all three types of nodes to our graph
g.add_node("router_agent", RunnableLambda(router_agent))     # Decision maker
g.add_node("email_tool",   RunnableLambda(email_tool))   # Data retriever  
g.add_node("chat_agent",   RunnableLambda(chat_agent))       # Response generator

print("✓ Added 3 nodes: router_agent, email_tool, chat_agent")

# === STEP 3: SET ENTRY POINT ===
# Always start with the router to make intelligent decisions
g.set_entry_point("router_agent")
print("✓ Set router_agent as entry point")

# === STEP 4: DEFINE ROUTE SELECTOR FUNCTION ===
def route_selector(state: AgentState) -> str:
    """
    Parse the router's decision and return the target node name.
    
    Args:
        state: Current agent state containing router's JSON decision
        
    Returns:
        str: Node name ("chat" or "email")
    """
    decision = json.loads(state["output"])
    tool_choice = decision.get("tool", "chat")  # Default to chat if no tool specified
    print(f"Route selector: directing to '{tool_choice}'")
    return tool_choice

# === STEP 5: ADD CONDITIONAL EDGES ===
# The router's output determines which path to take
g.add_conditional_edges(
    "router_agent",        # Source node
    route_selector,        # Function that decides the route
    {
        "chat":  "chat_agent",    # Direct to chat for simple queries
        "email": "email_tool",    # Route through email tool for email queries
    },
)
print("✓ Added conditional edges from router_agent")

# === STEP 6: ADD SIMPLE EDGES ===
# Email tool always hands off to chat agent for final response
g.add_edge("email_tool", "chat_agent")
print("✓ Added edge: email_tool → chat_agent")

# Both paths end at the chat agent, which then terminates
g.add_edge("chat_agent", END)
print("✓ Added edge: chat_agent → END")

# === STEP 7: COMPILE THE GRAPH ===
assistant_graph = g.compile()
print("✓ Graph compiled successfully!")

print("\nGraph structure:")
print("START → router_agent → [decision]")
print("                         ├─ 'chat' → chat_agent → END")
print("                         └─ 'email' → email_tool → chat_agent → END")
print("\nThe tool-enabled agent is ready!")

### 3.2 Interactive Tool-Enabled Chat

**Enhanced Chat Experience:**

This chat loop demonstrates the complete agent workflow:
1. **Router Analysis**: Determines if tools are needed
2. **Tool Execution**: Retrieves relevant data when required  
3. **Intelligent Response**: Combines conversation context with tool data
4. **State Management**: Clears tool context after each interaction

In [ ]:
# === INITIALIZE ENHANCED CONVERSATION STATE ===
chat_history = [
    {"role": "system", "content": "You are a helpful AI Agent. You have access to an email database if needed."}
]

state = AgentState(
    chat_history=chat_history,
    output=None,
    # Define available tools for the router to choose from
    available_tools={
        "email": "Search your recent e-mail archive", 
        "chat": "Continue regular conversation without tools"
    },
    tool_context=None  # Will be populated by tools when needed
)

print("🤖 Tool-enabled AI Agent initialized!")
print("Available tools: email search, regular chat")
print("Type 'exit' to quit the conversation.\n")

# === MAIN ENHANCED CHAT LOOP ===
while True:
    # === GET USER INPUT ===
    user_text = input("You: ").strip()
    
    # === CHECK FOR EXIT ===
    if user_text.lower() == "exit":
        break
    
    if not user_text:
        continue
    
    print(f"\n📝 Processing: '{user_text}'")
    
    # === ADD USER MESSAGE TO CONVERSATION ===
    state["chat_history"].append({"role": "user", "content": user_text})
    
    # === EXECUTE THE ENHANCED GRAPH ===
    # This will:
    # 1. Route through router_agent to decide on tools
    # 2. Optionally use email_tool to get context
    # 3. Generate final response with chat_agent
    print("🔄 Running enhanced agent graph...")
    state = assistant_graph.invoke(state)
    
    # === CLEANUP TOOL CONTEXT ===
    # Clear tool context after each interaction to prevent carryover
    state["tool_context"] = None
    
    # === DISPLAY RESPONSE ===
    print(f"🤖 Assistant: {state['output']}")
    print("-" * 60)

## Summary: Tool-Enabled AI Agent

**What We Built:**
A sophisticated AI agent that can intelligently decide when and how to use external tools to enhance its responses.

**Key Components:**

1. **Router Agent**: Makes intelligent decisions about tool usage
2. **Email Tool**: Retrieves contextual information from external sources  
3. **Enhanced Chat Agent**: Combines conversation with tool-provided context
4. **Conditional Graph**: Routes requests based on intelligent analysis

**Architecture Highlights:**

**Smart Routing:** The agent doesn't just blindly use tools - it analyzes each query to determine if tools are needed.

**Clean Separation:** Each component has a single responsibility:
- Router: Decision making
- Tools: Data retrieval  
- Chat: Response generation

**Context Management:** Tool context is added temporarily and cleaned up after each interaction.

**Scalable Design:** Easy to add new tools by:
- Writing new tools
- Updating the `available_tools` dictionary
- Adding new conditional routes

**Real-World Applications:**
- Email/document search and analysis
- Database queries with natural language
- API integrations for external services
- Multi-step workflows with tool chaining

**Next Steps:**
- Add more sophisticated tools (web search, calculations, file operations)
- Implement tool chaining (using multiple tools in sequence)
- Add conversation memory and user preferences
- Build specialized agents for different domains

**Congratulations!** 🎉 You now have a foundation for building production-ready AI agents with external tool capabilities.

## 4 - A Nod to MCP

MCP stands for *Model Context Protocol*. A modern, increasingly adopted standard for enabling AI agents to connect with and invoke external tools.
Think of MCP as a centralized hub: all the tools an AI model can access are hosted on an MCP server. Instead of embedding tool logic directly into the agent, the model simply makes API calls to the MCP server, which handles the heavy lifting. This approach offers several advantages:

 - Centralized management of tools and services
 - Improved monitoring and logging of tool usage
 - Offloading computation to dedicated systems, keeping the model's flow clean and efficient
 - Scalability and flexibility for enterprise-grade deployments

Here's an example of the same agent we built earlier, but this time it's connected to an MCP server I spun up just for this course.
You'll see how the agent's logic stays simple, while the MCP handles the tool orchestration behind the scenes.

### 4.1 Imports

In [ ]:
import truststore
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

### 4.2 Useful Functions and Setup

In [ ]:
# Setup needed for Michelin Netwrok workarounds
for k in ("SSL_CERT_FILE", "SSL_CERT_DIR", "REQUESTS_CA_BUNDLE"):
    os.environ.pop(k, None)
truststore.inject_into_ssl()

# URL of where the MCP Server is hosted
MCP_URL = "https://dev.d0s.michelin.com/projects/goaats_mcp/v1/mcp"
# Bearer Token
TOKEN = "dev-token-change-me"

# A function to list all tools available for this token on the MCP server
async def list_tools():
    client = MultiServerMCPClient(
        connections={
            "server": {
                "url": MCP_URL,
                "transport": "streamable_http",
                "headers": {"Authorization": f"Bearer {TOKEN}"},
            }
        }
    )
    async with client.session("server") as session:
        tools = await load_mcp_tools(session)
        return tools

# A function to call a specific tool by name
async def call_tool(name: str, arguments: dict):
    client = MultiServerMCPClient(
        connections={
            "server": {
                "url": MCP_URL,
                "transport": "streamable_http",
                "headers": {"Authorization": f"Bearer {TOKEN}"},
            }
        }
    )
    async with client.session("server") as session:
        tools = await load_mcp_tools(session)
        tool = next((t for t in tools if t.name == name), None)
        if not tool:
            return f"Tool {name!r} not found. Available: {[t.name for t in tools]}"
        return await tool.ainvoke(arguments)


### 4.3 Example Usages

#### 4.3.1 Listing Available Tools
A quick demo of how we can store and call multiple tools from the server

In [ ]:
tools = await list_tools()  # Example usage
for tool in tools:
    print("----------------------------------------")
    print("Name:", tool.name,"\n")
    print("Description:", tool.description,"\n")
    print("Tool Arguments:")
    if len(tool.args_schema["properties"].keys()) == 0:
        print("\tNone")
    else:
        for argument in tool.args_schema["properties"].keys():
            arg_props = tool.args_schema["properties"][argument]
            print("\tkey:", argument)
            print("\tName:", arg_props['title'])
            print("\tType:", arg_props['type'])
            print("\tRequired:", argument in tool.args_schema["required"])
            print("")
            
        

#### 4.3.2 Calling Specific Tool
A quick demo of actually calling the `math_evaluator` tool, to evaluate a written expression for us

In [ ]:
response = await call_tool("math_evaluator", {"expression": "100*(1 + 0.04)^5"})
# Response often comes as string, or list of strings, need to jsonify it
response = json.loads(response)
print("Input to Function:\t\t", response['input'])
print("Normalized expression:\t\t", response['normalized'])
print("Evaluated expression:\t\t", response['value'])

### 4.4 Incorporating MCP Into our Agent

#### 4.3.1 - Redefining Email Tool

In [ ]:
# Note that it's "async" now, as it has API calls
async def mcp_email_tool(state):
    print("\n--- MCP EMAIL TOOL NODE ---")

    email_archive = await call_tool("get_mock_emails", {}) # Calling MCP Function
    email_archive = [json.loads(email) for email in email_archive]

    context = "\n\n".join(
        f"""From: {email['sender']}
        Subject: {email['subject']}
        Body: {email['body']}"""
    for email in email_archive)

    print(f"Email archive context: {context[:100]}...")
    state["tool_context"] = context
    state["output"] = "email_context_ready"

    print("\n--- MCP EMAIL TOOL NODE END ---")

    return state

#### 4.3.2 - Redefining graph with our new tool

In [ ]:
# === STEP 1: INITIALIZE GRAPH ===
g = StateGraph(AgentState)

g.add_node("router_agent", RunnableLambda(router_agent)) 
g.add_node("email_tool",   RunnableLambda(mcp_email_tool))   # <-- Calling MCP email tool now
g.add_node("chat_agent",   RunnableLambda(chat_agent))

g.add_edge("email_tool", "chat_agent")
g.add_edge("chat_agent", END)

def route_selector(state: AgentState) -> str:
    decision = json.loads(state["output"])
    tool_choice = decision.get("tool", "chat")
    return tool_choice

g.add_conditional_edges(
    "router_agent",
    route_selector,
    {
        "chat":  "chat_agent",
        "email": "email_tool",
    },
)

g.set_entry_point("router_agent")

assistant_graph = g.compile()

#### 4.3.3 - Chat Loop with MCP Tools

In [ ]:
# === INITIALIZE ENHANCED CONVERSATION STATE ===
chat_history = [
    {"role": "system", "content": "You are a helpful AI Agent. You have access to an email database if needed."}
]

state = AgentState(
    chat_history=chat_history,
    output=None,
    available_tools={
        "email": "Search your recent e-mail archive", 
        "chat": "Continue regular conversation without tools"
    },
    tool_context=None
)

print("🤖 Tool-enabled AI Agent initialized!")
print("Available tools: email search, regular chat")
print("Type 'exit' to quit the conversation.\n")

while True:
    user_text = input("You: ").strip()
    
    if user_text.lower() == "exit":
        break
    
    if not user_text:
        continue
    
    print(f"\n📝 Processing: '{user_text}'")

    state["chat_history"].append({"role": "user", "content": user_text})
    
    print("🔄 Running enhanced agent graph...")
    state = await assistant_graph.ainvoke(state) # <--- Only change for asynchronous execution

    state["tool_context"] = None
    
    print(f"🤖 Assistant: {state['output']}")
    print("-" * 60)